# Setup OCI and Development Environment

This notebook walks you through the steps needed to setup your OCI tenancy and local workstation for Oracle Functions.

## Prerequisites

The following software **must** be installed:

1. Docker
1. Python 3


## Install the OCI Command Line Interface (CLI)

The following command installs the OCI CLI with the all the defaults:

In [ ]:
bash -c "$(curl -L https://raw.githubusercontent.com/oracle/oci-cli/master/scripts/install/install.sh)" -- --accept-all-defaults

## Install Fn CLI


In [ ]:
!curl -LSs https://raw.githubusercontent.com/fnproject/cli/master/install | sh

## Setup OCI Environment

1. Create Compartment `Kscope21`.
1. Create Virtual Cloud Network (VCN) `VCN-Kscope21` using the wizard with one public and one private subnet.
1. Create IAM Group `FunctionsDevelopers`.
1. Create IAM Policy `Fn-Root-Policy` with the following statements:
    ```
    Allow group FunctionsDevelopers to manage repos in tenancy
    Allow group FunctionsDevelopers to read objectstorage-namespaces in tenancy
    Allow group FunctionsDevelopers to use virtual-network-family in compartment Kscope21
    Allow group FunctionsDevelopers to manage functions-family in compartment Kscope21
    Allow group FunctionsDevelopers to read metrics in compartment Kscope21
    Allow group FunctionsDevelopers to use apm-domains in compartment Kscope21
    Allow service faas to use apm-domains in compartment Kscope21
    ```
1. Create IAM User `fndev`.
1. Add user to group `FunctionsDevelopers`.
1. Generate API Keys for user.
1. Generate Auth Token for user.
1. Gather information from OCI and enter them into the file `kscope21.env` and load the environment variables.



In [ ]:
. kscope21.env

Test again that the OCI CLI is correctly configured.

In [ ]:
oci os ns get

### Buckets and Resource Principal

The Function that will be created, requires access to the Object Storage buckets. For Resource Principal authentication, we need to create a Dynamic Group that includes the Functions created in the target compartment.

1. Create the Object Storage buckets `photos` and `thumbnails`.
1. Ensure that the `photos` bucket *Emit Object Events* attribute is enabled.
1. Get the compartment OCID and create the `FaasDynamicGroup` dynamic group, using the matching rule:
    ```
    All { resource.type = 'fnfunc', resource.compartment.id = '<compartment-ocid>' }
    ```
1. Create the `ImagesBucketAccessPolicy` that include the following two statements:
    ```
    allow dynamic-group FaasDynamicGroup to read buckets in compartment Kscope21
    allow dynamic-group FaasDynamicGroup to manage objects in compartment Kscope21 where any { target.bucket.name='photos', target.bucket.name='thumbnails' }
    ```

## Configure Fn Context

The following FN CLI commands will perform the following:

1. Create a new context and set it as the active context.
1. Set the required context parameters:
    * `oracle.profile`
    * `oracle.compartment-id`
    * `api-url`
    * `registry`

In [ ]:
fn create context $FN_CONTEXT_NAME --provider oracle && \
fn use context $FN_CONTEXT_NAME && \
fn update context oracle.profile $OCI_CLI_PROFILE && \
fn update context oracle.compartment-id $COMPARTMENT_OCID && \
fn update context api-url "https://functions.$REGION_ID.oci.oraclecloud.com" && \
fn update context registry "$REGION_KEY.ocir.io/$TENANCY_NAMESPACE/$APP_NAME"

Check that all the required parameters were set correctly:

In [ ]:
fn inspect context

Check that the FN CLI has been configured correctly and can communicate with the OCI tenancy:

In [ ]:
fn list apps

## Login to OCIR

Login to the OCIR using the format: 

```bash
docker login <region-key>.ocir.io 
```

> **Note.** The region key can be obtained from this [page](https://docs.oracle.com/en-us/iaas/Content/General/Concepts/regions.htm) in the OCI documentation.

And the username should be in the following format: 

```
<tenancy-namespace>/<username> 
```

In [ ]:
docker logout && \
docker login $REGION_KEY.ocir.io --username $DOCKER_USERNAME --password $DOCKER_PASSWORD